# 📊 User Behavior & Content Consumption Pattern Analysis
## End-to-End Movie Recommendation System - EDA

This notebook performs comprehensive Exploratory Data Analysis (EDA) on movie data to understand:
- **User Behavior Patterns**: How users interact with different genres, ratings, and content
- **Content Consumption Trends**: Popular genres, release patterns, budget vs revenue analysis
- **Recommendation Insights**: Data-driven insights for improving recommendation algorithms

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import json

warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported successfully!")

## 1️⃣ Data Loading & Initial Exploration

In [ ]:
# Load the dataset
df = pd.read_csv('./Artifacts/movies.csv')

print(f"📦 Dataset Shape: {df.shape}")
print(f"📋 Total Movies: {len(df):,}")
print(f"📊 Total Features: {df.shape[1]}")
print("\n" + "="*50)
print("\n📌 Column Names:")
print(df.columns.tolist())

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Data Info
df.info()

In [ ]:
# Statistical Summary
df.describe()

## 2️⃣ Data Cleaning & Preprocessing

In [ ]:
# Check for missing values
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage': missing_percent
}).sort_values('Percentage', ascending=False)

print("🔍 Missing Data Analysis:")
print(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# Visualize missing data
plt.figure(figsize=(14, 6))
missing_cols = missing_df[missing_df['Missing Count'] > 0].head(10)
sns.barplot(x=missing_cols.index, y=missing_cols['Percentage'], palette='viridis')
plt.title('📊 Top 10 Columns with Missing Data', fontsize=16, fontweight='bold')
plt.xlabel('Columns', fontsize=12)
plt.ylabel('Missing Percentage (%)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 3️⃣ User Behavior Analysis - Rating Patterns

In [ ]:
# Rating Distribution Analysis
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
axes[0].hist(df['vote_average'].dropna(), bins=30, color='#e50914', edgecolor='black', alpha=0.7)
axes[0].set_title('📊 User Rating Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Average Rating', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].axvline(df['vote_average'].mean(), color='#FAFA33', linestyle='--', linewidth=2, label=f'Mean: {df["vote_average"].mean():.2f}')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box Plot
axes[1].boxplot(df['vote_average'].dropna(), vert=True, patch_artist=True,
                boxprops=dict(facecolor='#e50914', alpha=0.7),
                medianprops=dict(color='#FAFA33', linewidth=2))
axes[1].set_title('📦 Rating Distribution (Box Plot)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Average Rating', fontsize=12)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Rating Statistics:")
print(f"   Mean Rating: {df['vote_average'].mean():.2f}")
print(f"   Median Rating: {df['vote_average'].median():.2f}")
print(f"   Std Deviation: {df['vote_average'].std():.2f}")

In [ ]:
# Vote Count Analysis - User Engagement
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Log scale histogram
vote_counts = df['vote_count'].dropna()
axes[0].hist(np.log1p(vote_counts), bins=50, color='#17a2b8', edgecolor='black', alpha=0.7)
axes[0].set_title('📊 User Engagement (Log Scale)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Log(Vote Count + 1)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].grid(alpha=0.3)

# Scatter: Rating vs Vote Count
sample = df.sample(min(1000, len(df)))
axes[1].scatter(sample['vote_count'], sample['vote_average'], alpha=0.5, c='#e50914', s=30)
axes[1].set_title('🎯 Rating vs User Engagement', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Vote Count', fontsize=12)
axes[1].set_ylabel('Average Rating', fontsize=12)
axes[1].set_xscale('log')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 4️⃣ Content Consumption - Genre Analysis

In [ ]:
# Extract and analyze genres
from collections import Counter

# Parse genres
all_genres = []
for genres in df['genres'].dropna():
    genre_list = genres.split()
    all_genres.extend(genre_list)

genre_counts = Counter(all_genres)
top_genres = dict(genre_counts.most_common(15))

# Visualize
plt.figure(figsize=(14, 7))
bars = plt.barh(list(top_genres.keys()), list(top_genres.values()), color='#e50914', edgecolor='black')
plt.title('🎬 Top 15 Most Popular Genres', fontsize=16, fontweight='bold')
plt.xlabel('Number of Movies', fontsize=12)
plt.ylabel('Genre', fontsize=12)
plt.gca().invert_yaxis()

# Add value labels
for i, (genre, count) in enumerate(top_genres.items()):
    plt.text(count + 50, i, f'{count:,}', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n📊 Genre Statistics:")
print(f"   Total Unique Genres: {len(genre_counts)}")
print(f"   Most Popular Genre: {list(top_genres.keys())[0]} ({list(top_genres.values())[0]:,} movies)")

## 5️⃣ Temporal Analysis - Release Patterns

In [ ]:
# Parse release dates
df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year

# Year-wise movie count
year_counts = df['release_year'].value_counts().sort_index()
year_counts = year_counts[year_counts.index >= 1980]  # Filter from 1980 onwards

plt.figure(figsize=(16, 6))
plt.plot(year_counts.index, year_counts.values, color='#e50914', linewidth=2, marker='o', markersize=4)
plt.fill_between(year_counts.index, year_counts.values, alpha=0.3, color='#e50914')
plt.title('📅 Movie Release Trends Over Time (1980-Present)', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Movies Released', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📊 Release Statistics:")
print(f"   Peak Year: {year_counts.idxmax()} ({year_counts.max():,} movies)")
print(f"   Average Movies/Year: {year_counts.mean():.0f}")

## 6️⃣ Financial Analysis - Budget vs Revenue

In [ ]:
# Filter movies with valid budget and revenue
financial_df = df[(df['budget'] > 0) & (df['revenue'] > 0)].copy()
financial_df['roi'] = (financial_df['revenue'] - financial_df['budget']) / financial_df['budget'] * 100

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Budget vs Revenue Scatter
sample = financial_df.sample(min(1000, len(financial_df)))
axes[0, 0].scatter(sample['budget'], sample['revenue'], alpha=0.5, c='#e50914', s=30)
axes[0, 0].plot([0, sample['budget'].max()], [0, sample['budget'].max()], 'k--', alpha=0.3, label='Break-even')
axes[0, 0].set_title('💰 Budget vs Revenue', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Budget ($)', fontsize=12)
axes[0, 0].set_ylabel('Revenue ($)', fontsize=12)
axes[0, 0].set_xscale('log')
axes[0, 0].set_yscale('log')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# ROI Distribution
roi_filtered = financial_df[financial_df['roi'] < 1000]['roi']  # Filter outliers
axes[0, 1].hist(roi_filtered, bins=50, color='#17a2b8', edgecolor='black', alpha=0.7)
axes[0, 1].set_title('📊 Return on Investment (ROI) Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('ROI (%)', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].axvline(roi_filtered.median(), color='#FAFA33', linestyle='--', linewidth=2, label=f'Median: {roi_filtered.median():.0f}%')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)
# Budget Distribution
axes[1, 0].hist(np.log10(financial_df['budget']), bins=50, color='#28a745', edgecolor='black', alpha=0.7)
axes[1, 0].set_title('💵 Budget Distribution (Log Scale)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Log10(Budget)', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].grid(alpha=0.3)

# Revenue Distribution
axes[1, 1].hist(np.log10(financial_df['revenue']), bins=50, color='#ffc107', edgecolor='black', alpha=0.7)
axes[1, 1].set_title('💰 Revenue Distribution (Log Scale)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Log10(Revenue)', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💰 Financial Statistics:")
print(f"   Average Budget: ${financial_df['budget'].mean():,.0f}")
print(f"   Average Revenue: ${financial_df['revenue'].mean():,.0f}")
print(f"   Median ROI: {roi_filtered.median():.0f}%")

## 7️⃣ Popularity Analysis

In [ ]:
# Popularity Distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
axes[0].hist(df['popularity'].dropna(), bins=50, color='#e50914', edgecolor='black', alpha=0.7)
axes[0].set_title('📈 Popularity Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Popularity Score', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_xlim(0, df['popularity'].quantile(0.95))  # Remove outliers for better visualization
axes[0].grid(alpha=0.3)

# Top 20 Most Popular Movies
top_popular = df.nlargest(20, 'popularity')[['title', 'popularity', 'vote_average']]
axes[1].barh(range(20), top_popular['popularity'], color='#17a2b8', edgecolor='black')
axes[1].set_yticks(range(20))
axes[1].set_yticklabels([title[:30] + '...' if len(title) > 30 else title for title in top_popular['title']], fontsize=9)
axes[1].set_title('🔥 Top 20 Most Popular Movies', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Popularity Score', fontsize=12)
axes[1].invert_yaxis()
axes[1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 8️⃣ Runtime Analysis

In [ ]:
# Runtime Distribution
runtime_data = df['runtime'].dropna()
runtime_data = runtime_data[(runtime_data > 0) & (runtime_data < 300)]  # Filter outliers

plt.figure(figsize=(14, 6))
plt.hist(runtime_data, bins=50, color='#6f42c1', edgecolor='black', alpha=0.7)
plt.axvline(runtime_data.mean(), color='#FAFA33', linestyle='--', linewidth=2, label=f'Mean: {runtime_data.mean():.0f} min')
plt.axvline(runtime_data.median(), color='#e50914', linestyle='--', linewidth=2, label=f'Median: {runtime_data.median():.0f} min')
plt.title('⏱️ Movie Runtime Distribution', fontsize=16, fontweight='bold')
plt.xlabel('Runtime (minutes)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n⏱️ Runtime Statistics:")
print(f"   Average Runtime: {runtime_data.mean():.0f} minutes")
print(f"   Median Runtime: {runtime_data.median():.0f} minutes")
print(f"   Shortest Movie: {runtime_data.min():.0f} minutes")
print(f"   Longest Movie: {runtime_data.max():.0f} minutes")

## 9️⃣ Correlation Analysis

In [ ]:
# Select numeric columns for correlation
numeric_cols = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']
corr_data = df[numeric_cols].dropna()

# Calculate correlation matrix
correlation_matrix = corr_data.corr()

# Visualize
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('🔗 Feature Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n🔗 Key Correlations:")
print("   Budget ↔ Revenue:", f"{correlation_matrix.loc['budget', 'revenue']:.2f}")
print("   Vote Count ↔ Popularity:", f"{correlation_matrix.loc['vote_count', 'popularity']:.2f}")
print("   Vote Average ↔ Vote Count:", f"{correlation_matrix.loc['vote_average', 'vote_count']:.2f}")

## 🔟 Key Insights & Recommendations

In [ ]:
print("="*70)
print("🎯 KEY INSIGHTS FROM EDA")
print("="*70)

print("\n📊 1. USER BEHAVIOR PATTERNS:")
print(f"   • Average user rating: {df['vote_average'].mean():.2f}/10")
print(f"   • Most engaged movies have {df['vote_count'].quantile(0.75):.0f}+ votes")
print(f"   • Rating distribution is slightly left-skewed (more high ratings)")

print("\n🎬 2. CONTENT CONSUMPTION TRENDS:")
print(f"   • Top genre: {list(top_genres.keys())[0]} ({list(top_genres.values())[0]:,} movies)")
print(f"   • Average movie runtime: {runtime_data.mean():.0f} minutes")
print(f"   • Peak production year: {year_counts.idxmax()} ({year_counts.max():,} movies)")

print("\n💰 3. FINANCIAL INSIGHTS:")
print(f"   • Average budget: ${financial_df['budget'].mean():,.0f}")
print(f"   • Average revenue: ${financial_df['revenue'].mean():,.0f}")
print(f"   • Median ROI: {roi_filtered.median():.0f}%")

print("\n🚀 4. RECOMMENDATIONS FOR SYSTEM IMPROVEMENT:")
print("   ✓ Focus on popular genres for better recommendations")
print("   ✓ Consider user engagement (vote_count) as a quality signal")
print("   ✓ Weight recent movies higher in recommendations")
print("   ✓ Use budget/revenue data for commercial success prediction")
print("   ✓ Implement genre-based collaborative filtering")

print("\n" + "="*70)
print("✅ EDA COMPLETED SUCCESSFULLY!")
print("="*70)